In [1]:
import pandas as pd
import glob
import numpy as np
from scipy import stats
from sklearn import cluster
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score



In [2]:
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM

Using TensorFlow backend.


In [896]:
#path to celllines folder where I have 44 celllines
path = './celllines' # use your path
all_files = glob.glob(path + "/*.csv")


In [943]:
emb = pd.read_csv('./network_embedding.csv', index_col=None, header=0)

In [945]:
emb.set_index('53',inplace=True)

In [989]:
EGF=emb.loc['EGF'].values
EGFR=emb.loc['EGFR'].values
MEK=emb.loc['MEK12'].values
PI3K=emb.loc['PI3K'].values
PKC=emb.loc['PKC'].values
MTOR=emb.loc['mTOR'].values



In [1009]:
def prepare_data(all_files,EGF,EGFR,MEK,PI3K,PKC,MTOR):
    df_values=list()
    for i in range (len(all_files)):    #len(all_files)
        filename = all_files[i]

        print(filename)
        df = pd.read_csv(filename, index_col=None, header=0)
        # drop markers not found in test data
        df.drop(['p.HER2','p.PLCg2'], axis=1,inplace=True)
        # fill na
        df.fillna(method='ffill', inplace=True)
        # remove dublicate
        df["id"] = df["treatment"] + df["cell_line"] + df["time"].map(str) + df["cellID"].map(str)
        df = df.drop_duplicates(subset='id', keep="first")
        df.drop(['id'], axis=1,inplace=True)
        # set full time to -1
        df.at[df['treatment']=='full', 'time'] = -1
        # drop some times 
        indexNames =df[ df['time'] == 5.5 ].index
        indexNames2=df[ df['time'] == 23 ].index
        indexNames3=df[ df['time'] == 30 ].index

        # Delete these row indexes from dataFrame
        df.drop(indexNames , inplace=True)
        df.drop(indexNames2 , inplace=True)
        df.drop(indexNames3 , inplace=True)

        data=df.copy() #EGF,iEGFR,iMEK,iPI3K,iPKC
        data_EGF=data[data['treatment']=='EGF']
        data_EGF=data_EGF.append(data[data['treatment']=='full'])
        #keep only cells with all 8 time steps
        data_EGF=data_EGF.groupby('cellID').filter(lambda x : len(x)==8)
        if not data_EGF.empty:
            data_EGF.drop(['treatment','cell_line','fileID'], axis=1,inplace=True)
            data_EGF.set_index('cellID',inplace=True)
            data_EGF_sort= data_EGF.sort_values(['cellID', 'time'], ascending=[True, True])
            data_EGF_sort.drop(['time'], axis=1,inplace=True)
            #to put all time steps in one raw 
            df_EGF = data_EGF_sort.groupby(['cellID'])['b.CATENIN', 'cleavedCas', 'CyclinB', 'GAPDH', 'IdU', 'Ki.67',
               'p.4EBP1', 'p.Akt.Ser473.', 'p.AKT.Thr308.', 'p.AMPK', 'p.BTK',
               'p.CREB', 'p.ERK', 'p.FAK', 'p.GSK3b', 'p.H3', 'p.JNK',
               'p.MAP2K3', 'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6', 'p.MKK4', 'p.NFkB',
               'p.p38', 'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.RB', 'p.S6',
               'p.S6K', 'p.SMAD23', 'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5'].apply(lambda x: pd.DataFrame(x.values, columns=x.columns))
            df_EGF=df_EGF.unstack()#.sort_index(axis=1, level=1)
            m=np.repeat([EGF], df_EGF.shape[0], axis=0)
            concat=np.concatenate((m, df_EGF.values), axis=1)
            #print(concat)
            print(concat.shape)
            df_values.extend(concat)
       

        data_iEGFR=data[data['treatment']=='iEGFR']
        data_iEGFR=data_iEGFR.append(data[data['treatment']=='full'])
        #keep only cells with all 8 time steps
        data_iEGFR=data_iEGFR.groupby('cellID').filter(lambda x : len(x)==8)
        if not data_iEGFR.empty:
            data_iEGFR.drop(['treatment','cell_line','fileID'], axis=1,inplace=True)
            data_iEGFR.set_index('cellID',inplace=True)
            data_iEGFR_sort= data_iEGFR.sort_values(['cellID', 'time'], ascending=[True, True])
            data_iEGFR_sort.drop(['time'], axis=1,inplace=True)
            df_iEGFR = data_iEGFR_sort.groupby(['cellID'])['b.CATENIN', 'cleavedCas', 'CyclinB', 'GAPDH', 'IdU', 'Ki.67',
               'p.4EBP1', 'p.Akt.Ser473.', 'p.AKT.Thr308.', 'p.AMPK', 'p.BTK',
               'p.CREB', 'p.ERK', 'p.FAK', 'p.GSK3b', 'p.H3', 'p.JNK',
               'p.MAP2K3', 'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6', 'p.MKK4', 'p.NFkB',
               'p.p38', 'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.RB', 'p.S6',
               'p.S6K', 'p.SMAD23', 'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5'].apply(lambda x: pd.DataFrame(x.values, columns=x.columns))
            df_iEGFR=df_iEGFR.unstack()#.sort_index(axis=1, level=1)
            m=np.repeat([EGFR], df_iEGFR.shape[0], axis=0)
            concat=np.concatenate((m, df_iEGFR.values), axis=1)
            #print(concat)
            print(concat.shape)
            df_values.extend(concat)
        

        data_iMEK=data[data['treatment']=='iMEK']
        data_iMEK=data_iMEK.append(data[data['treatment']=='full'])
        #keep only cells with all 8 time steps
        data_iMEK=data_iMEK.groupby('cellID').filter(lambda x : len(x)==8)
        if not data_iMEK.empty:
            data_iMEK.drop(['treatment','cell_line','fileID'], axis=1,inplace=True)
            data_iMEK.set_index('cellID',inplace=True)
            data_iMEK_sort= data_iMEK.sort_values(['cellID', 'time'], ascending=[True, True])
            data_iMEK_sort.drop(['time'], axis=1,inplace=True)
            df_iMEK= data_iMEK_sort.groupby(['cellID'])['b.CATENIN', 'cleavedCas', 'CyclinB', 'GAPDH', 'IdU', 'Ki.67',
               'p.4EBP1', 'p.Akt.Ser473.', 'p.AKT.Thr308.', 'p.AMPK', 'p.BTK',
               'p.CREB', 'p.ERK', 'p.FAK', 'p.GSK3b', 'p.H3', 'p.JNK',
               'p.MAP2K3', 'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6', 'p.MKK4', 'p.NFkB',
               'p.p38', 'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.RB', 'p.S6',
               'p.S6K', 'p.SMAD23', 'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5'].apply(lambda x: pd.DataFrame(x.values, columns=x.columns))
            df_iMEK=df_iMEK.unstack()#.sort_index(axis=1, level=1)
            m=np.repeat([MEK], df_iMEK.shape[0], axis=0)
            concat=np.concatenate((m, df_iMEK.values), axis=1)
            #print(concat)
            print(concat.shape)
            df_values.extend(concat)
        


        data_iPI3K=data[data['treatment']=='iPI3K']
        data_iPI3K=data_iPI3K.append(data[data['treatment']=='full'])
        #keep only cells with all 8 time steps
        data_iPI3K=data_iPI3K.groupby('cellID').filter(lambda x : len(x)==8)
        if not data_iPI3K.empty:
            data_iPI3K.drop(['treatment','cell_line','fileID'], axis=1,inplace=True)
            data_iPI3K.set_index('cellID',inplace=True)
            data_iPI3K_sort= data_iPI3K.sort_values(['cellID', 'time'], ascending=[True, True])
            data_iPI3K_sort.drop(['time'], axis=1,inplace=True)
            df_iPI3K= data_iPI3K_sort.groupby(['cellID'])['b.CATENIN', 'cleavedCas', 'CyclinB', 'GAPDH', 'IdU', 'Ki.67',
               'p.4EBP1', 'p.Akt.Ser473.', 'p.AKT.Thr308.', 'p.AMPK', 'p.BTK',
               'p.CREB', 'p.ERK', 'p.FAK', 'p.GSK3b', 'p.H3', 'p.JNK',
               'p.MAP2K3', 'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6', 'p.MKK4', 'p.NFkB',
               'p.p38', 'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.RB', 'p.S6',
               'p.S6K', 'p.SMAD23', 'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5'].apply(lambda x: pd.DataFrame(x.values, columns=x.columns))
            df_iPI3K=df_iPI3K.unstack()#.sort_index(axis=1, level=1)
            m=np.repeat([PI3K], df_iPI3K.shape[0], axis=0)
            concat=np.concatenate((m, df_iPI3K.values), axis=1)
            #print(concat)
            print(concat.shape)
            df_values.extend(concat)
       

        data_iPKC=data[data['treatment']=='iPKC']
        data_iPKC=data_iPKC.append(data[data['treatment']=='full'])
        #keep only cells with all 8 time steps
        data_iPKC=data_iPKC.groupby('cellID').filter(lambda x : len(x)==8)
        if not data_iPKC.empty:
            data_iPKC.drop(['treatment','cell_line','fileID'], axis=1,inplace=True)
            data_iPKC.set_index('cellID',inplace=True)
            data_iPKC_sort= data_iPKC.sort_values(['cellID', 'time'], ascending=[True, True])
            data_iPKC_sort.drop(['time'], axis=1,inplace=True)
            df_iPKC= data_iPKC_sort.groupby(['cellID'])['b.CATENIN', 'cleavedCas', 'CyclinB', 'GAPDH', 'IdU', 'Ki.67',
               'p.4EBP1', 'p.Akt.Ser473.', 'p.AKT.Thr308.', 'p.AMPK', 'p.BTK',
               'p.CREB', 'p.ERK', 'p.FAK', 'p.GSK3b', 'p.H3', 'p.JNK',
               'p.MAP2K3', 'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6', 'p.MKK4', 'p.NFkB',
               'p.p38', 'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.RB', 'p.S6',
               'p.S6K', 'p.SMAD23', 'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5'].apply(lambda x: pd.DataFrame(x.values, columns=x.columns))
            df_iPKC=df_iPKC.unstack()#.sort_index(axis=1, level=1)

            m=np.repeat([PKC], df_iPKC.shape[0], axis=0)
            concat=np.concatenate((m, df_iPKC.values), axis=1)
            #print(concat)
            print(concat.shape)
            df_values.extend(concat)
        
        
    return(df_values)

    

In [1010]:
data=prepare_data(all_files,EGF,EGFR,MEK,PI3K,PKC,MTOR)

./celllines/MFM223.csv
(6969, 296)
(4187, 296)
(4659, 296)
(6962, 296)
(4202, 296)
./celllines/MCF7.csv
(7155, 296)
(5195, 296)
(1000, 296)
(6535, 296)
(4793, 296)
./celllines/MDAMB175VII.csv
(2883, 296)
(2222, 296)
(1113, 296)
(2831, 296)
(170, 296)
./celllines/HCC3153.csv
(5722, 296)
(6033, 296)
(4106, 296)
(4472, 296)
(2977, 296)
./celllines/HCC1395.csv
(6082, 296)
(5268, 296)
(4471, 296)
(3250, 296)
./celllines/HBL100.csv
(3821, 296)
(3627, 296)
(3438, 296)
(4287, 296)
(4287, 296)
./celllines/DU4475.csv
(803, 296)
(1946, 296)
(2155, 296)
(1256, 296)
(1197, 296)
./celllines/MDAkb2.csv
(5315, 296)
(3823, 296)
(3775, 296)
(2901, 296)
./celllines/CAL148.csv
(3649, 296)
(3760, 296)
(3550, 296)
./celllines/T47D.csv
(6215, 296)
(3562, 296)
(3850, 296)
(4294, 296)
(3889, 296)
./celllines/HDQP1.csv
(6942, 296)
(6476, 296)
(5807, 296)
(6942, 296)
(3737, 296)
./celllines/HCC2157.csv
(6307, 296)
(5123, 296)
(4111, 296)
(5056, 296)
./celllines/HCC1500.csv
(4692, 296)
(2501, 296)
(3717, 296)
(18

In [1015]:
train=np.asarray(data)

In [1016]:
train.shape

(988929, 296)

In [1017]:
np.save('train_emb.npy', train)    # .npy extension is added if not given
#train= np.load('train.npy')

In [1018]:
train.shape

(988929, 296)

In [1235]:
# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons, timesteps):
    n_lag=35+16
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    #X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X.shape[1],), activation='relu'))
    #model.add(Dense(neurons, activation='relu'))
    model.add(Dense(y.shape[1], activation='relu'))
    #model.add(Dropout(0.5))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X, y, epochs=nb_epoch, batch_size=batch_size, verbose=1,validation_split=0.2, shuffle=False)
    
    return model

    

In [1236]:
# fit model

model = fit_lstm(train, 32, 3, 64, 1)

    

Train on 791143 samples, validate on 197786 samples
Epoch 1/3
791143/791143 [==============================] - 220s 278us/step - loss: 0.9578 - val_loss: 1.1943
Epoch 2/3
791143/791143 [==============================] - 245s 310us/step - loss: 0.7275 - val_loss: 1.1758
Epoch 3/3
791143/791143 [==============================] - 244s 309us/step - loss: 0.7291 - val_loss: 1.1630


In [1237]:
################TESt
#path to subchallenge folder where I have 12 celllines 
#'184B5' 'BT483' 'HCC1428' 
#'HCC1806' 'HCC202' 'Hs578T' 
#'MCF12A' 'MDAMB231' 'MDAMB468'
#'SKBR3' 'UACC3199' 'ZR751'
path = './sub2' # use your path
test_files = [path + "/184B5.csv",path + "/BT483.csv", path + "/HCC1428.csv",
             path + "/HCC1806.csv",path + "/HCC202.csv",  path + "/Hs578T.csv",
             path + "/MCF12A.csv",path + "/MDAMB231.csv", path + "/MDAMB468.csv",
             path + "/SKBR3.csv",path + "/UACC3199.csv",  path + "/ZR751.csv"]


In [1238]:
def prepare_data(all_files):
    df_values=list()
    for i in range (len(all_files)):    #len(all_files)
        filename = all_files[i]

        print(filename)
        df = pd.read_csv(filename, index_col=None, header=0)
        # drop markers not found in test data
        df.drop(['p.HER2','p.PLCg2'], axis=1,inplace=True)
        # fill na
        df.fillna(method='ffill', inplace=True)
        # set full time to -1
        df.at[df['treatment']=='full', 'time'] = -1
        inputdata=df[df['treatment']=='full']
        inputdata.drop(['treatment','cell_line','fileID','time'], axis=1,inplace=True)
        inputdata.set_index('cellID',inplace=True)
        inputdata=inputdata[:10000]
        if inputdata.shape[0]<10000:
            inputdata= inputdata.append(inputdata)
            inputdata= inputdata.append(inputdata)
        inputdata=inputdata[:10000]
        #print(inputdata.shape)
        name=filename.split('/')[2].split('.')[0]
        if (name =='BT483' or name == 'MCF12A' or name == 'MDAMB468' ):
            m=np.repeat([EGFR], 10000, axis=0)
            concat=np.concatenate((m, inputdata.values), axis=1)
            #print(concat)
            print(concat.shape)
            df_values.extend(concat)
            
        if (name =='184B5' or name == 'ZR751' or name == 'HCC202' ):
            m=np.repeat([MEK], 10000, axis=0)
            concat=np.concatenate((m, inputdata.values), axis=1)
            #print(concat)
            print(concat.shape)
            df_values.extend(concat)
            
        if (name =='UACC3199' or name == 'SKBR3' or name == 'MDAMB231' ):
            m=np.repeat([PI3K], 10000, axis=0)
            concat=np.concatenate((m, inputdata.values), axis=1)
            #print(concat)
            print(concat.shape)
            df_values.extend(concat)
            
        if (name =='HCC1806' or name == 'Hs578T' or name == 'HCC1428' ):
            m=np.repeat([PI3K], 10000, axis=0)
            concat=np.concatenate((m, inputdata.values), axis=1)
            #print(concat)
            print(concat.shape)
            df_values.extend(concat)

    return df_values


In [1239]:
data=prepare_data(test_files)

./sub2/184B5.csv
(10000, 51)
./sub2/BT483.csv
(10000, 51)
./sub2/HCC1428.csv
(10000, 51)
./sub2/HCC1806.csv
(10000, 51)
./sub2/HCC202.csv
(10000, 51)
./sub2/Hs578T.csv
(10000, 51)
./sub2/MCF12A.csv
(10000, 51)
./sub2/MDAMB231.csv
(10000, 51)
./sub2/MDAMB468.csv
(10000, 51)
./sub2/SKBR3.csv
(10000, 51)
./sub2/UACC3199.csv
(10000, 51)
./sub2/ZR751.csv
(10000, 51)


In [1240]:
test=np.asarray(data)

In [1241]:
test.shape

(120000, 51)

In [1242]:
# make a prediction
n_lag=35+16
X, y = test[:, 0:n_lag], test[:, n_lag:]
print(X.shape)
#X = X.reshape(X.shape[0], 1, X.shape[1])
yhat = model.predict(X)




(120000, 51)


In [1243]:
# invert scaling
#yhat = invert_scale(scaler, X, yhat)

In [1244]:
tempdata=yhat.reshape(yhat.shape[0]*7,35)

In [1245]:
###### TEMP

#path to cellline 
#MCF12A miss time 13
filename = './subchallenge_2_template_data.csv' # use your path

print(filename)
df_temp = pd.read_csv(filename, index_col=None, header=0)

./subchallenge_2_template_data.csv


In [1246]:
dfpred=pd.DataFrame(tempdata)

In [1247]:
# drop time 13 in cell MC...
indx= np.arange(450000,450000+10000)
# Delete these row indexes from dataFrame
dfpred.drop(indx , inplace=True)

In [1248]:
dfpred.shape

(830000, 35)

In [1249]:
dfpred.insert (0, "cell_line", df_temp.cell_line.values)

In [1250]:
dfpred.insert (1, "treatment", df_temp.treatment.values)

In [1251]:
dfpred.insert (2, "time", df_temp.time.values)

In [1252]:
dfpred.insert (3, "cellID", df_temp.cellID.values)

In [1253]:
dfpred.columns = df_temp.columns

In [1254]:
dfpred

,cell_line,treatment,time,cellID,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,Ki.67,...,p.p90RSK,p.PDPK1,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
0,184B5,iMEK,0,1,5.555454,5.446057,8.196292,5.307769,6.206110,2.972082,...,4.221492,3.334708,2.499907,3.004896,2.223133,2.696813,1.752462,3.167734,3.982780,2.572870
1,184B5,iMEK,0,2,2.285702,2.974255,2.989305,3.438920,2.707711,3.610435,...,1.187446,2.529655,2.465488,1.714642,3.428814,3.683746,2.627208,2.266058,2.826346,2.019090
2,184B5,iMEK,0,3,2.086494,1.495960,2.410165,3.044149,2.396289,2.033992,...,1.425909,2.405695,3.331073,2.771612,2.016547,2.910561,1.464634,1.944766,1.176771,2.205042
3,184B5,iMEK,0,4,2.926005,2.170706,1.628857,2.199518,2.080055,2.324231,...,2.686518,1.969670,2.915853,3.198473,4.467035,3.125852,4.639481,5.921216,5.281863,3.932868
4,184B5,iMEK,0,5,5.356617,2.915166,3.323148,2.704105,3.670446,4.498660,...,3.183716,3.646550,2.756717,4.376248,5.311630,3.978230,3.350405,4.387887,1.556624,2.261532
5,184B5,iMEK,0,6,1.315764,2.467791,3.290939,2.266021,1.876386,-0.000000,...,2.658545,3.287683,2.623525,2.028854,2.720202,1.617546,2.243225,1.301926,2.413225,3.273135
6,184B5,iMEK,0,7,2.221238,1.831687,2.480966,2.109715,2.364300,1.831585,...,1.707648,1.925915,2.586385,2.493125,2.350683,2.963866,3.359869,3.042423,2.729214,2.869335
7,184B5,iMEK,0,8,6.273668,5.558699,7.259284,7.053840,6.936877,1.974669,...,2.963136,2.582736,1.958259,2.301064,1.546133,2.129422,1.510014,2.016910,2.268464,1.562500
8,184B5,iMEK,0,9,1.246886,1.779451,1.989283,2.720493,2.198365,2.497990,...,0.734294,1.754493,1.851630,1.239139,2.205389,2.049543,1.550693,1.273497,1.693324,1.465588
9,184B5,iMEK,0,10,1.594136,1.346051,1.637471,1.827714,1.555311,1.263128,...,1.206299,1.493908,1.771849,1.521032,1.107400,1.715541,0.930544,1.435138,0.937588,1.314300


In [1255]:
dfpred.to_csv(r'filled2_subchallenge_2_template_data.csv')

In [1256]:
#        'BT20', 'BT474', 'BT549', 'CAL148', 'CAL51', 'CAL851', 'DU4475',
#       'EFM192A', 'EVSAT', 'HBL100', 'HCC1187', 'HCC1395', 'HCC1419',
#       'HCC1500', 'HCC1569', 'HCC1599', 'HCC1937', 'HCC1954', 'HCC2185',
#       'HCC3153', 'HCC38', 'HCC70', 'HDQP1', 'JIMT1', 'MCF7',
#       'MDAMB134VI', 'MDAMB157', 'MDAMB175VII', 'MDAMB361', 'MDAMB415',
#       'MDAMB453', 'MFM223', 'MPE600', 'MX1', 'OCUBM', 'T47D', 'UACC812',
#       'UACC893', 'ZR7530'

path = './celllines' # use your path
test_files = [path + "/BT20.csv",path + "/BT474.csv", path + "/BT549.csv",path + "/CAL148.csv",
              path + "/CAL51.csv", path + "/CAL851.csv",
              path + "/DU4475.csv",path + "/EFM192A.csv",path + "/EVSAT.csv",path + "/HBL100.csv",path + "/HCC1187.csv",
             path + "/HCC1395.csv",path + "/HCC1419.csv",  path + "/HCC1500.csv",path + "/HCC1569.csv",path + "/HCC1599.csv",
             path + "/HCC1937.csv",path + "/HCC1954.csv", path + "/HCC2185.csv",path + "/HCC3153.csv",path + "/HCC38.csv",
             path + "/HCC70.csv",path + "/HDQP1.csv",  path + "/JIMT1.csv",path + "/MCF7.csv",path + "/MDAMB134VI.csv",
              path +'/MDAMB157.csv', path +'/MDAMB175VII.csv',path + '/MDAMB361.csv',path + '/MDAMB415.csv', path + '/MDAMB453.csv',
             path + "/MFM223.csv",path + "/MPE600.csv",path + "/MX1.csv",path + "/OCUBM.csv",path + "/T47D.csv",
             path + "/UACC812.csv",path + "/UACC893.csv",path + "/ZR7530.csv"]

In [1257]:
def prepare_data(all_files):
    df_values=list()
    for i in range (len(all_files)):    #len(all_files)
        filename = all_files[i]

        print(filename)
        df = pd.read_csv(filename, index_col=None, header=0)
        # drop markers not found in test data
        df.drop(['p.HER2','p.PLCg2'], axis=1,inplace=True)
        # fill na
        df.fillna(method='ffill', inplace=True)
        # set full time to -1
        df.at[df['treatment']=='full', 'time'] = -1
        inputdata=df[df['treatment']=='full']
        inputdata.drop(['treatment','cell_line','fileID','time'], axis=1,inplace=True)
        inputdata.set_index('cellID',inplace=True)
        inputdata=inputdata[:10000]
        if inputdata.shape[0]<10000:
            inputdata= inputdata.append(inputdata)
            inputdata= inputdata.append(inputdata)
            inputdata= inputdata.append(inputdata)
        inputdata=inputdata[:10000]
        print(inputdata.shape)
        
        
        m=np.repeat([MTOR], 10000, axis=0)
        concat=np.concatenate((m, inputdata.values), axis=1)
        #print(concat)
        print(concat.shape)
        df_values.extend(concat)
            

    return df_values


In [1258]:
data=prepare_data(test_files)

./celllines/BT20.csv
(10000, 35)
(10000, 51)
./celllines/BT474.csv
(10000, 35)
(10000, 51)
./celllines/BT549.csv
(10000, 35)
(10000, 51)
./celllines/CAL148.csv
(10000, 35)
(10000, 51)
./celllines/CAL51.csv
(10000, 35)
(10000, 51)
./celllines/CAL851.csv
(10000, 35)
(10000, 51)
./celllines/DU4475.csv
(10000, 35)
(10000, 51)
./celllines/EFM192A.csv
(10000, 35)
(10000, 51)
./celllines/EVSAT.csv
(10000, 35)
(10000, 51)
./celllines/HBL100.csv
(10000, 35)
(10000, 51)
./celllines/HCC1187.csv
(10000, 35)
(10000, 51)
./celllines/HCC1395.csv
(10000, 35)
(10000, 51)
./celllines/HCC1419.csv
(10000, 35)
(10000, 51)
./celllines/HCC1500.csv
(10000, 35)
(10000, 51)
./celllines/HCC1569.csv
(10000, 35)
(10000, 51)
./celllines/HCC1599.csv
(10000, 35)
(10000, 51)
./celllines/HCC1937.csv
(10000, 35)
(10000, 51)
./celllines/HCC1954.csv
(10000, 35)
(10000, 51)
./celllines/HCC2185.csv
(10000, 35)
(10000, 51)
./celllines/HCC3153.csv
(10000, 35)
(10000, 51)
./celllines/HCC38.csv
(10000, 35)
(10000, 51)
./celllin

In [1261]:
test=np.asarray(data)

In [1262]:
# make a prediction
n_lag=35+16
X, y = test[:, 0:n_lag], test[:, n_lag:]
print(X.shape)
#X = X.reshape(X.shape[0], 1, X.shape[1])
yhat = model.predict(X)

(390000, 51)


In [1263]:
tempdata=yhat.reshape(yhat.shape[0]*7,35)

In [1264]:
dfpred3=pd.DataFrame(tempdata)

In [1265]:
###### TEMP

#path to cellline 
#MDAMB175VII miss time 60
filename = './subchallenge_3_template_data.csv' # use your path

print(filename)
df_temp3 = pd.read_csv(filename, index_col=None, header=0)

./subchallenge_3_template_data.csv


In [1266]:
df_temp3

,cell_line,treatment,time,cellID,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,Ki.67,...,p.p90RSK,p.PDPK1,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
0,BT20,imTOR,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BT20,imTOR,0,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BT20,imTOR,0,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BT20,imTOR,0,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BT20,imTOR,0,5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BT20,imTOR,0,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BT20,imTOR,0,7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BT20,imTOR,0,8,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,BT20,imTOR,0,9,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BT20,imTOR,0,10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1267]:

# drop time 60 in cell MDAMB175VII...
indx= np.arange(1950000,1950000+10000)
# Delete these row indexes from dataFrame
dfpred3.drop(indx , inplace=True)


In [1268]:
dfpred3.insert (0, "cell_line", df_temp3.cell_line.values)
dfpred3.insert (1, "treatment", df_temp3.treatment.values)
dfpred3.insert (2, "time", df_temp3.time.values)
dfpred3.insert (3, "cellID", df_temp3.cellID.values)

In [1269]:
dfpred3.columns = df_temp3.columns

In [1270]:
dfpred3.to_csv(r'filled2_subchallenge_3_template_data.csv')

In [1271]:
dfpred3

,cell_line,treatment,time,cellID,b.CATENIN,cleavedCas,CyclinB,GAPDH,IdU,Ki.67,...,p.p90RSK,p.PDPK1,p.RB,p.S6,p.S6K,p.SMAD23,p.SRC,p.STAT1,p.STAT3,p.STAT5
0,BT20,imTOR,0,1,3.548095,3.195194,5.370129,2.819929,3.325144,3.255570,...,3.327139,2.815834,2.264157,2.524974,2.083365,2.082300,1.676408,2.864308,3.592168,2.812313
1,BT20,imTOR,0,2,2.838480,3.152849,2.973599,2.881333,2.916024,3.450615,...,1.248916,2.319033,2.153587,2.215144,3.331746,3.505127,2.954642,2.942407,3.136576,1.783881
2,BT20,imTOR,0,3,1.772192,1.436716,2.173197,2.823277,2.519313,2.326181,...,1.500698,2.295409,3.120879,3.098364,2.626381,3.075912,1.421916,1.635012,1.379786,2.133930
3,BT20,imTOR,0,4,2.807379,2.462983,2.130620,2.356324,2.054768,2.005448,...,3.130252,2.674415,3.128980,3.169827,3.546093,3.313759,4.198934,5.391961,5.264076,4.416804
4,BT20,imTOR,0,5,5.051187,2.869174,2.725522,2.759450,3.381418,4.202901,...,2.809932,2.838257,2.964284,4.101100,4.891491,4.231136,3.875296,4.434796,1.568564,1.925433
5,BT20,imTOR,0,6,1.569594,2.433638,3.194256,2.634703,2.551717,-0.000000,...,2.445148,3.100945,2.782233,2.433626,2.733426,1.480938,1.830716,1.354888,2.292509,3.057508
6,BT20,imTOR,0,7,2.497636,2.377745,2.672509,1.849964,1.968199,1.719449,...,1.977167,1.942287,2.356747,1.949912,2.106977,2.560458,3.049200,2.811530,2.731436,2.633768
7,BT20,imTOR,0,8,3.592690,4.290864,5.370301,2.591109,3.689386,4.150320,...,3.886485,3.530273,2.749322,3.003954,2.487831,2.873041,1.812018,3.620770,4.267176,3.772204
8,BT20,imTOR,0,9,3.607374,3.945590,3.650393,3.780398,3.350791,4.292307,...,1.549462,2.812918,2.927294,2.474715,4.175679,4.188299,3.870175,3.728887,3.890110,2.096865
9,BT20,imTOR,0,10,2.363926,1.442986,2.709268,3.355465,3.244532,2.919250,...,1.653474,2.931735,3.733347,4.092130,3.396466,3.912007,1.734834,2.297667,1.513155,2.744833
